In [1]:
from jax_circuits import*
from topology import *
from penalty import *
from cp_utils import *
import matplotlib.pyplot as plt
from scipy.stats import unitary_group
from qiskit import transpile
from qiskit.quantum_info import Operator
import pickle
import time

import hyperopt
from hyperopt import hp, fmin, tpe, Trials, STATUS_OK

import hyperopt.pyll.stochastic
from hyperopt.pyll import scope

In [57]:
qc = QuantumCircuit(3)
qc.ccx(0, 1, 2)
u_toff3 = Operator(qc.reverse_bits()).data

num_samples=50

def objective(args):
    r, num_gates = args
    anz = Ansatz(3, 'cp', placements=fill_layers(chain_layer(3), num_gates))
    reg_options = {'r': r,
               'function': 'linear',
               'ymax': 2,
               'xmax': jnp.pi/2,
               'plato': 0.05,
               'accepted_num_gates': 12,
               'target_num_gates': 8,
               'cp_mask': anz.cp_mask}
    
    score, cz_counts = cp_ansatz_score(u_toff3, 
                                       anz, 
                                       reg_options, 
                                       num_samples=num_samples,
                                       save_prospective_results=False,
                                       save_raw_results=False,
                                       cp_dist='0')
    
    return {
        'loss': -score,
        'status': STATUS_OK}

In [58]:
space = [hp.loguniform('r', jnp.log(0.0001), jnp.log(0.01)),
        scope.int(hp.quniform('num_gates', 1, 30, 1))]

samples = [hyperopt.pyll.stochastic.sample(space) for _ in range(100)] 
r_samples = [r for r,_ in samples]
g_samples = [g for _,g in samples]

# plt.xscale('log')
# plt.hist(r_samples)

In [59]:
trials = Trials()
best = fmin(objective,
    space=space,
    algo=tpe.suggest,
    max_evals=150,
    trials=trials)

print(best)

100%|██████████| 150/150 [1:09:03<00:00, 27.62s/trial, best loss: -16.0625]
{'num_gates': 21.0, 'r': 0.0005310848219253889}


In [60]:
qc = QuantumCircuit(3)
qc.ccx(0, 1, 2)
u_toff3 = Operator(qc.reverse_bits()).data

anz = Ansatz(3, 'cp', fill_layers(chain_layer(3), 21))

reg_options = {'r': 0.0005310848219253889,
               'function': 'linear',
               'ymax': 2,
               'xmax': jnp.pi/2,
               'plato': 0.05,
               'target_num_gates': 8,
               'accepted_num_gates': 8,
               'angle_tolerance': 0.2,
               'cp_mask': anz.cp_mask}

key = random.PRNGKey(2)
num_samples = 50

successful_results, failed_results = cp_decompose(u_toff3, 
                                               anz,
                                               regularization_options=reg_options,
                                               num_samples=num_samples, 
                                               key=key,
                                               disc_func=disc2,
                                               cp_dist='0',
                                               save_to='toff3_connected')


Computing raw results.

Selecting prospective results:


50it [00:00, 75.01it/s]


4 found.

Verifying prospective results:


100%|██████████| 4/4 [01:46<00:00, 26.54s/it]


4 successful.
cz counts are:
[8, 8, 8, 8]


In [61]:
qc = QuantumCircuit(3)
qc.ccx(0, 1, 2)
u_toff3 = Operator(qc.reverse_bits()).data

num_samples=50

def objective(args):
    r, num_gates = args
    anz = Ansatz(3, 'cp', placements=fill_layers(chain_layer(3), num_gates))
    reg_options = {'r': r,
               'function': 'linear',
               'ymax': 2,
               'xmax': jnp.pi/2,
               'plato': 0.05,
               'accepted_num_gates': 12,
               'target_num_gates': 8,
               'cp_mask': anz.cp_mask}
    
    score, cz_counts = cp_ansatz_score(u_toff3, 
                                       anz, 
                                       reg_options, 
                                       num_samples=num_samples,
                                       save_prospective_results=False,
                                       save_raw_results=False,
                                       cp_dist='uniform')
    
    return {
        'loss': -score,
        'status': STATUS_OK}

space = [hp.loguniform('r', jnp.log(0.0001), jnp.log(0.01)),
        scope.int(hp.quniform('num_gates', 1, 30, 1))]

trials = Trials()
best = fmin(objective,
    space=space,
    algo=tpe.suggest,
    max_evals=100,
    trials=trials)

print(best)

100%|██████████| 100/100 [42:08<00:00, 25.28s/trial, best loss: -25.25] 
{'num_gates': 13.0, 'r': 0.000644899814208608}


In [11]:
qc = QuantumCircuit(3)
qc.ccx(0, 1, 2)
u_toff3 = Operator(qc.reverse_bits()).data

anz = Ansatz(3, 'cp', fill_layers(chain_layer(3), 13))

reg_options = {'r': 0.0002510848219253889,
               'function': 'linear',
               'ymax': 2,
               'xmax': jnp.pi/2,
               'plato': 0.05,
               'target_num_gates': 8,
               'accepted_num_gates': 8,
               'angle_tolerance': 0.2,
               'cp_mask': anz.cp_mask}

key = random.PRNGKey(5)
num_samples = 100

successful_results, failed_results = cp_decompose(u_toff3, 
                                               anz,
                                               regularization_options=reg_options,
                                               num_samples=num_samples, 
                                               key=key,
                                               disc_func=disc2,
                                               cp_dist='uniform',
                                               save_to='toff3_connected')


Computing raw results.

Selecting prospective results:


100it [00:01, 89.37it/s]


13 found.

Verifying prospective results:


100%|██████████| 13/13 [04:39<00:00, 21.51s/it]


13 successful.
cz counts are:
[8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8]


In [2]:
qc = QuantumCircuit(4)
qc.mct([0, 1, 2], 3)
u_toff4 = Operator(qc.reverse_bits()).data

num_samples=200

def objective(args):
    r, num_gates = args
    anz = Ansatz(4, 'cp', placements=fill_layers(chain_layer(4), num_gates))
    reg_options = {'r': r,
               'function': 'linear',
               'ymax': 2,
               'xmax': jnp.pi/2,
               'plato': 0.05,
               'accepted_num_gates': 25,
               'target_num_gates': 18,
               'cp_mask': anz.cp_mask}
    
    score, cz_counts = cp_ansatz_score(u_toff4, 
                                       anz, 
                                       reg_options, 
                                       num_samples=num_samples,
                                       save_prospective_results=False,
                                       save_raw_results=False,
                                       cp_dist='uniform')
    
    return {
        'loss': -score,
        'status': STATUS_OK}

space = [hp.loguniform('r', jnp.log(0.00005), jnp.log(0.001)),
        scope.int(hp.quniform('num_gates', 15, 40, 1))]

trials = Trials()
best = fmin(objective,
    space=space,
    algo=tpe.suggest,
    max_evals=50,
    trials=trials)

print(best)    

100%|██████████| 50/50 [1:39:57<00:00, 119.95s/trial, best loss: -1.2890625]
{'num_gates': 25.0, 'r': 0.0005908527276785403}


In [3]:
time_start = time.time()
qc = QuantumCircuit(4)
qc.mct([0, 1, 2], 3)
u_toff4 = Operator(qc.reverse_bits()).data


anz = Ansatz(4, 'cp', fill_layers(chain_layer(4), 25))

reg_options = {'r': 0.0005908527276785403,
               'function': 'linear',
               'ymax': 2,
               'xmax': jnp.pi/2,
               'plato': 0.05,
               'accepted_num_gates': 19,
               'target_num_gates': 18,
               'cp_mask': anz.cp_mask}

key = random.PRNGKey(131)

num_samples = 2000

successful_results, failed_results = cp_decompose(u_toff4, 
                                               anz,
                                               regularization_options=reg_options,
                                               num_samples=num_samples, 
                                               key=key,
                                               disc_func=disc2,
                                               cp_dist='uniform',
                                               save_to='toff4_chain')

print(f'time: {time.time()-time_start}')


Computing raw results.

Selecting prospective results:


2000it [00:39, 50.52it/s]


1 found.

Verifying prospective results:


100%|██████████| 1/1 [00:47<00:00, 47.34s/it]

1 successful.
cz counts are:
[18]
time: 1090.0114843845367
